In [ ]:
import   sqlhelper
import   utils

from selenium.webdriver.common.action_chains import ActionChains
import requests

import urllib.request
import re
import os
import urllib
from requests_html import HTMLSession
import time
import pymysql
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import mysql.connector
from lxml import etree
import requests

ch_op = Options()
    #设置谷歌浏览器的页面无可视化
ch_op.add_argument('--headless')
ch_op.add_argument('--disable-gpu')
ch_op.add_argument('blink-settings=imagesEnabled=false')  # 不加载图片, 提升速度
ch_op.add_experimental_option('excludeSwitches', ['enable-automation'])
driver = webdriver.Chrome(chrome_options=ch_op)


def htmlspecialchars(text):
    return (
        text.replace("&", "&amp;").
        replace('"', "&quot;").
        replace("<", "&lt;").
        replace(">", "&gt;")
    )
#过滤HTML中的标签
# 将HTML中标签等信息去掉
# @param htmlstr HTML字符串.
def filter_tags(htmlstr):
    # 先过滤CDATA
    re_cdata = re.compile('//<!\[CDATA\[[^>]*//\]\]>', re.I)  # 匹配CDATA
    re_script = re.compile('<\s*script[^>]*>[^<]*<\s*/\s*script\s*>', re.I)  # Script
    re_style = re.compile('<\s*style[^>]*>[^<]*<\s*/\s*style\s*>', re.I)  # style
    re_br = re.compile('<br\s*?/?>')  # 处理换行
    re_h = re.compile('</?\w+[^>]*>')  # HTML标签
    re_comment = re.compile('<!--[^>]*-->')  # HTML注释
    s = re_cdata.sub('', htmlstr)  # 去掉CDATA
    s = re_script.sub('', s)  # 去掉SCRIPT
    s = re_style.sub('', s)  # 去掉style
    s = re_br.sub('\n', s)  # 将br转换为换行
    s = re_h.sub('', s)  # 去掉HTML 标签
    s = re_comment.sub('', s)  # 去掉HTML注释
    # 去掉多余的空行
    blank_line = re.compile('\n+')
    s = blank_line.sub('\n', s)
    s = replaceCharEntity(s)  # 替换实体
    return s


##替换常用HTML字符实体.
# 使用正常的字符替换HTML中特殊的字符实体.
# 你可以添加新的实体字符到CHAR_ENTITIES中,处理更多HTML字符实体.
# @param htmlstr HTML字符串.
def replaceCharEntity(htmlstr):
    CHAR_ENTITIES = {'nbsp': ' ', '160': ' ',
                     'lt': '<', '60': '<',
                     'gt': '>', '62': '>',
                     'amp': '&', '38': '&',
                     'quot': '"', '34': '"', }

    re_charEntity = re.compile(r'&#?(?P<name>\w+);')
    sz = re_charEntity.search(htmlstr)
    while sz:
        entity = sz.group()  # entity全称，如&gt;
        key = sz.group('name')  # 去除&;后entity,如&gt;为gt
        try:
            htmlstr = re_charEntity.sub(CHAR_ENTITIES[key], htmlstr, 1)
            sz = re_charEntity.search(htmlstr)
        except KeyError:
            # 以空串代替
            htmlstr = re_charEntity.sub('', htmlstr, 1)
            sz = re_charEntity.search(htmlstr)
    return htmlstr


def repalce(s, re_exp, repl_string):
    return re_exp.sub(repl_string, s)


def dir_file_list(path):
    file_list = []
    import os
    for root,dirs,files in os.walk(path):
        file_list = file_list+files
    return file_list


def remove_dir(path,print_result = False):
    import os
    existed=os.path.exists(path)
    if existed == False:
        message = 'Result: '+path+' not existed'
    else:
        file_list = dir_file_list(path)
        file_count = len(file_list)
        for file_name in file_list:
            os.unlink(path+'/'+file_name)
        os.rmdir(path)
        message = 'Result: '+path+' already removed with ' + str(file_count) + ' files in it'
    if print_result:
        print(message)

#根据给定的网址来获取网页详细信息，得到的html就是网页的源代码  
def getHtml(url):
    page = urllib.request.urlopen(url)
    html = page.read()
    return html.decode('UTF-8')

def getTitleImg(url,id):
    path = 'f:\\phpstudy_pro\\WWW\\images\\' +str(id)+'\\' 
   # 将图片保存到D:\\test文件夹中，如果没有test文件夹则创建
    if not os.path.isdir(path):  
        os.makedirs(path)  
    paths = path+'\\'      #保存在test路径下 
    x="imgtitle"+str(id)
    imgurl="http:"+url
    ext=(imgurl.rpartition("."))[2]
    print(ext) 
    print("titleimgurl=",imgurl)
    urllib.request.urlretrieve(imgurl,'{0}{1}.{2}'.format(paths,x,ext))
    return  imgurl
    
def getImg(html,id):
   # reg = r'src="(.+?\.jpg)" pic_ext'
   # <img(?<Attributes1>[\s\S]*?)src=("{1}|'{1}|)(?<picture>[^\[^>]*?(gif|jpg|jpeg|bmp|bmp))("{1}|'{1}|)(?<Attributes2>[\s\S]*?)>
  
    #reg=r'//.+?(.jpg|.png|gif)'
    #reg=r'//\S+(.jpg|.png|gif)'
    reg=r'(?=//)\S+?\.jpg|(?=//)\S+?\.gif|(?=//)\S+?\.png|(?=//)\S+?\.bmp'
    #reg=r'(?<=//).+?(.jpg|.png|gif)'
   # reg=r'//.+?.jpg'
    imgre = re.compile(reg)
    imglist = imgre.findall(html)#表示在整个网页中过滤出所有图片的地址，放在imglist中
    x = 1
    path = 'f:\\phpstudy_pro\\WWW\\images\\' +str(id)+'\\' 
   # 将图片保存到D:\\test文件夹中，如果没有test文件夹则创建
    if not os.path.isdir(path):  
        os.makedirs(path)  
    paths = path+'\\'      #保存在test路径下 
    print(imglist)
    for imgurl in imglist: 
        print(imgurl)
    img_b_e=[]
    for imgurl in imglist:  
        imgurl="http:"+imgurl
        print(imgurl)
        print("+++++++++++++++++++++++++++++++++++++")
        ext=(imgurl.rpartition("."))[2]
        print(ext) 
        print("imgurl=",imgurl)
        imgurl=imgurl.replace("////","//")
        urllib.request.urlretrieve(imgurl,'{0}{1}.{2}'.format(paths,x,ext))  #打开imglist中保存的图片网址，并下载图片保存在本地，format格式化字符串 
     #   x = x + 1 
        tag="@$!tag"+str(x)+"@$!"
        imgurl=imgurl.replace("http:","")
        img_b_e.append((imgurl,'/images/{2}/{0}.{1}'.format(x,ext,str(id)),tag))
        x = x + 1 
    return img_b_e

def gettopimgs(imglist,id,tag):
      path = 'f:\\phpstudy_pro\\WWW\\images\\' +str(id)+'\\' 
   # 将图片保存到D:\\test文件夹中，如果没有test文件夹则创建
      if not os.path.isdir(path):  
        os.makedirs(path)  
      paths = path+'\\'      #保存在test路径下 
      print(imglist)
      for imgurl in imglist: 
           print(imgurl)
      img_b_e=[]   
      x = 1
      for imgurl in imglist:  
           imgurl="http:"+imgurl
           print(imgurl)
           print("+++++++++++++++++++++++++++++++++++++")
           ext=(imgurl.rpartition("."))[2]
           print(ext) 
           print("imgurl=",imgurl)
           imgurl=imgurl.replace("////","//")
           name=tag+str(x)
           urllib.request.urlretrieve(imgurl,'{0}{1}.{2}'.format(paths,name,ext))  #设置s表示小图片，打开imglist中保存的图片网址，并下载图片保存在本地，format格式化字符串 

        #imgurl=imgurl.replace("http:","")
           img_b_e.append('/images/{2}/{0}.{1}'.format(name,ext,str(id)))
           x = x + 1            
      return img_b_e


sql="SELECT * FROM spider WHERE  tag IS NULL"
queryresult=sqlhelper.select_all(sql,[])

print(len(queryresult))
def mouseMoveOnClick(self, element):
        """
        鼠标悬停事件出现菜单,并点击
        element要点击的元素位置
        """
        script = '$(arguments[0]).click()'
        self.driver.js(script, element)

for x in queryresult:
   print(x[2])
   imgtitleurl=x[2]
   id=x[0]
   print("id=",id)
   print(x[7])
   url="https:"+x[7]

   try:
     if "item.x.com" in url:
   
      # driver = webdriver.Chrome(chrome_options=ch_op)
       driver.get(url)  
       driver.implicitly_wait(5) # <div class="p-parameter">
       js = "q=document.documentElement.scrollTop=1903" # 将页面滚动条往下拖
       driver.execute_script(js)
       driver.implicitly_wait(4) 
       element=driver.find_element_by_xpath("//div[@class='p-parameter']").text
       myDynamicElement = driver.find_element_by_xpath("//div[@id='J-detail-content']").get_attribute("outerHTML")
        
       if "商品介绍加载中" in myDynamicElement :
              time.sleep(random.randint(900,1500))   
              continue
       else:
              pagesource=driver.page_source
    
              tree = etree.HTML(pagesource)
        
              toppicsurl=tree.xpath("//div[@class='spec-items']/ul[@class='lh']/li/img/@src")
              print("toppicsurl:************************:",toppicsurl)
              tag="s"
              toppic_s=""
              toppic_m=""
              toppic_b=""
       
              for picurl in gettopimgs(toppicsurl,id,tag):
                       picurl="<img src='"+picurl+"'>";
                       toppic_s=toppic_s+picurl
       # piclist=driver.find_elements_by_xpath("//div[@id='spec-list']/ul/li/img")      
              piclist_b=[]
              piclist_m=[]
              for mouse in driver.find_elements_by_xpath("//div[@id='spec-list']/div/ul/li/img"):
                       ActionChains(driver).move_to_element(mouse).perform()  #悬停失效
                       #time.sleep(1)
                       print("悬停检测")
                     #  driver.execute_script('$(arguments[0]).click()',mouse)
                       pichtml=driver.find_element_by_id("spec-n1").get_attribute("outerHTML")
                     #  pichtml=driver.find_element_by_class_name("jqzoom main-img").get_attribute("outerHTML")
                       print("pichtml*************************:",pichtml)
                       print(pichtml)
                       # htmlsource=driver.page_source
    
                       tree = etree.HTML(pichtml)
        
                       toppicsurl_b=tree.xpath("//img/@jqimg")
                         
                       toppicsurl_m=tree.xpath("//img/@src")
                       piclist_b.append(toppicsurl_b[0])
                       piclist_m.append(toppicsurl_m[0])
              tag="b"
              print("piclist_b:",piclist_b)
      # print(gettopimgs(piclist_b,id,tag))

    #   print(gettopimgs(piclist_m,id,tag))  
              for picurl in gettopimgs(piclist_b,id,tag):
                       picurl="<img src='"+picurl+"'>";
                       toppic_b=toppic_b+picurl
              tag="m"           
              for picurl in gettopimgs(piclist_m,id,tag):
                       picurl="<img src='"+picurl+"'>";
                       toppic_m=toppic_m+picurl
        
              print(toppic_s)
              print(toppic_b)
              print(toppic_m)

              myDynamicElement =myDynamicElement.replace("//misc.360buyimg.com/lib/img/e/blank.gif", "")                                
              print(myDynamicElement)

             # break
     #  myDynamicElement2= re.sub(r'<[^>]+>', '', myDynamicElement) #去掉html标签，主要检测内容全是背景图设置的页面
              newimg=getImg(myDynamicElement,id)
              getTitleImg(imgtitleurl,id)
    
              reg=r'background-image:url\S+?\.png|background-image:url\S+?\.jpg'
              imgre = re.compile(reg)
              backgroundimglist = imgre.findall(myDynamicElement)
              print("************************************")
              print(backgroundimglist)
              backgroundimg=[]
              for img in    backgroundimglist:
                    for  img2 in newimg:
                          if img2[0].replace("http:","") in img:
                                  backgroundimg.append(img2[1])  #新图片地址
                    
            
      # print(".............................................")
      # print(backgroundimg)
    
       #print(".............................................")     

              reg=r'(<img.+?>)'  #查找<img>标签

              imgre = re.compile(reg)
              imglist = imgre.findall(myDynamicElement)
              print("img datalazy img 列表：",imglist)
              print("newimg 标签：",newimg)
              for img in imglist:
                   for   img2 in newimg:
                        if img2[0] in img:
                              myDynamicElement=myDynamicElement.replace(img, img2[2])  #原文img 转换成标签
                              print(img)
                              print(".............................................")
                              print(img2[2])
                            
              print("原文img 转换成标签:", myDynamicElement)                 
              re_style = re.compile('<\s*style[^>]*>[^<]*<\s*/\s*style\s*>', re.I)  # style
              myDynamicElement= re.sub(re_style, '', myDynamicElement)  #清理详解部分原文的style
              myDynamicElement= re.sub(r'<[^>]+>', '', myDynamicElement)  #清空原文htnml标签
    
              for   img2 in newimg:
              
                       myDynamicElement=myDynamicElement.replace(img2[2], "<img src='"+img2[1]+"'>")  #把插入的标签置换成图片
              print("原文标签 转换成img:", myDynamicElement)  
              for  img in backgroundimg:
                       myDynamicElement=myDynamicElement+"<br><img src='"+img+"'>"
    
     #  print(myDynamicElement)
              content=element+myDynamicElement
              print(content)
              #content="s"
              content=content.replace("\n","</br>")
              content=htmlspecialchars(content)  #对特殊字符转义
              print(content)
              titleimgurl="/images/"+str(id)+"/imgtitle"+str(id)+".jpg"   
             # driver.close()
            #  sql = "UPDATE spider SET tag='Y' ,content=\""+content+"\" ,img='"+titleimgurl+"'  WHERE id ="+str(id)
              print(sql)
              sql = "UPDATE spider SET tag = %s,content=%s,img=%s,topimgs=%s,topimgm=%s,topimgb=%s ,date2=%s  WHERE id= %s"
              time1=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())
              booleantag=sqlhelper.update_one(sql,['Y',content,titleimgurl,toppic_s,toppic_m,toppic_b,time1,str(id)])
              print("booleantag=",booleantag)
            # sql = "insert into stu(id,name,age) VALUE (%s,%s,%s)"   #增加
            # res = insert_one(sql, [2,"飞机", 28]) 
           #   path = 'C:\\phpstudy_pro\\WWW\\images\\' +str(id)+'\\' 
           #   remove_dir(path,print_result=True)
  
   except:
      delpath = 'f:\\phpstudy_pro\\WWW\\images\\' +str(id)+'\\' 
      remove_dir(delpath,print_result=True)
      time.sleep(random.randint(15,110))
      continue
            
   time.sleep(random.randint(15,110))   
driver.close()
myresult.close()
mydb.close()
